In [1]:
import sys
import os

src_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
if src_path not in sys.path:
    sys.path.append(src_path)

In [2]:
from src.core.llm_generators.dashboards import MetabaseDashboardGenerator
from src.config.prompts import prompts
from src.config.settings import settings

In [3]:
dash_gen = MetabaseDashboardGenerator(
    metabase_url=settings.METABASE_URL,
    username=settings.METABASE_USERNAME,
    password=settings.METABASE_PASSWORD
)

In [19]:
import yaml
with open("../artifacts/deploy/dbt/models/marts/schema.yml", "r", encoding="utf-8") as f:
    schema = yaml.safe_load(f)

In [15]:
with open("../artifacts/analytics_spec.yml", "r", encoding="utf-8") as f:
    spec = yaml.safe_load(f)

In [18]:
metrics = spec["metrics"]

In [20]:
r = dash_gen.generate_cards_data(marts_schema=schema,
                                 metrics=metrics)

In [24]:
[n for n, k in r.items()]

['total_sales_by_day', 'unique_customers_per_region', 'average_order_value']

In [25]:
cards_ids = []
for card_name, card_data in r.items():
    new_card_id = dash_gen.create_card(name=card_name, 
                                visualization_settings=card_data['visualization_settings'],
                                query=card_data['query'],
                                display=card_data['display'])
    cards_ids.append(new_card_id)

In [26]:
cards_ids

[67, 68, 69]

In [27]:
# создание дашборда
dashboard_id = dash_gen.create_dashboard(name="Analytics Dashboard")


In [28]:
dash_gen.add_cards_to_dashboard(dashboard_id=dashboard_id,
                                cards_ids=cards_ids)
        

{'description': '',
 'archived': False,
 'view_count': 0,
 'collection_position': None,
 'dashcards': [{'size_x': 12,
   'dashboard_tab_id': None,
   'series': [],
   'action_id': None,
   'collection_authority_level': None,
   'card': {'cache_invalidated_at': None,
    'description': None,
    'archived': False,
    'view_count': 0,
    'collection_position': None,
    'source_card_id': None,
    'table_id': None,
    'result_metadata': None,
    'initially_published_at': None,
    'can_write': True,
    'database_id': 2,
    'enable_embedding': False,
    'collection_id': None,
    'query_type': 'native',
    'name': 'total_sales_by_day',
    'last_used_at': '2025-05-22T17:09:34.729817Z',
    'type': 'question',
    'creator_id': 1,
    'moderation_reviews': [],
    'updated_at': '2025-05-22T17:09:34.729817Z',
    'made_public_by_id': None,
    'embedding_params': None,
    'cache_ttl': None,
    'dataset_query': {'type': 'native',
     'database': 2,
     'native': {'query': 'SELECT

In [8]:
dash_gen.add_cards_to_dashboard(dashboard_id=33,
                                cards_ids=[33, 65, 66])

{'description': '',
 'archived': False,
 'view_count': 60,
 'collection_position': None,
 'dashcards': [{'size_x': 12,
   'dashboard_tab_id': None,
   'series': [],
   'action_id': None,
   'collection_authority_level': None,
   'card': {'cache_invalidated_at': None,
    'description': None,
    'archived': False,
    'view_count': 7,
    'collection_position': None,
    'source_card_id': None,
    'table_id': None,
    'result_metadata': [{'display_name': 'date_day',
      'field_ref': ['field', 'date_day', {'base-type': 'type/Date'}],
      'base_type': 'type/Date',
      'effective_type': 'type/Date',
      'name': 'date_day',
      'semantic_type': None,
      'fingerprint': {'global': {'distinct-count': 4, 'nil%': 0.0},
       'type': {'type/DateTime': {'earliest': '2023-10-01T00:00:00Z',
         'latest': '2023-10-04T00:00:00Z'}}}},
     {'display_name': 'total_sales',
      'field_ref': ['field', 'total_sales', {'base-type': 'type/Float'}],
      'base_type': 'type/Float',
    

In [ ]:
import requests
from typing import Dict, Any, List

class MetabaseDashboardGenerator:
    def __init__(self, metabase_url: str, username: str, password: str):
        self.metabase_url = metabase_url.rstrip('/')
        self.username = username
        self.password = password
        self.session_id = self._authenticate()

    def _authenticate(self) -> str:
        url = f"{self.metabase_url}/api/session"
        response = requests.post(url, json={
            "username": self.username,
            "password": self.password
        })
        response.raise_for_status()
        return response.json()['id']

    def _headers(self) -> Dict[str, str]:
        return {
            "Content-Type": "application/json",
            "X-Metabase-Session": self.session_id
        }
    
    def get_dashboard(self, dashboard_id: int):
        url = f"{self.metabase_url}/api/dashboard/{dashboard_id}"
        resp = requests.get(
            url,
            headers=self._headers()
        )
        dashboard_data = resp.json()
        return dashboard_data

    def create_card(self, name: str, visualization_settings: Dict[str, Any], query: Dict[str, Any], display: str = "table") -> int:
        """
        Создаёт карту (график или таблицу) в Metabase.
        :param name: Название карты
        :param database_id: ID базы данных в Metabase
        :param query: Словарь с параметрами запроса (native или simple)
        :param display: Тип отображения ('table', 'bar', 'line', 'pie' и т.д.)
        :return: ID созданной карты
        """
        url = f"{self.metabase_url}/api/card"
        payload = {
            "name": name,
            "dataset_query": query,
            "display": display,
            "visualization_settings": visualization_settings
        }
        response = requests.post(url, json=payload, headers=self._headers())
        response.raise_for_status()
        return response.json()['id']

    def create_dashboard(self, name: str, description: str = "") -> int:
        """
        Создаёт новый дашборд.
        :param name: Название дашборда
        :param description: Описание
        :return: ID дашборда
        """
        url = f"{self.metabase_url}/api/dashboard"
        payload = {
            "name": name,
            "description": description
        }
        response = requests.post(url, json=payload, headers=self._headers())
        response.raise_for_status()
        return response.json()['id']

    def add_card_to_dashboard(self, dashboard_id: int, card_id: int, sizeX: int = 4, sizeY: int = 4, row: int = 0, col: int = 0):
        """
        Добавляет карту на дашборд.
        :param dashboard_id: ID дашборда
        :param card_id: ID карты
        :param sizeX: ширина виджета
        :param sizeY: высота виджета
        :param row: строка
        :param col: столбец
        """
        dashboard_data = self.get_dashboard(dashboard_id)
        
        new_dashcard = {
            "id": card_id,
            "card_id": card_id,
            "col": col,
            "row": row,
            "size_x": sizeX,
            "size_y": sizeY,
            "series": [],
            "parameter_mappings": []
        }
        
        dashboard_data["dashcards"].append(new_dashcard)

        url = f"{self.metabase_url}/api/dashboard/{dashboard_id}"
        response = requests.put(url, json=dashboard_data, headers=self._headers())
        response.raise_for_status()
        return response.json()

    def create_dashboard_with_cards(self, dashboard_name: str, cards: List[Dict[str, Any]]) -> int:
        """
        Создаёт дашборд и добавляет на него несколько карт.
        :param dashboard_name: Название дашборда
        :param cards: Список словарей с параметрами для create_card
        :return: ID дашборда
        """
        dashboard_id = self.create_dashboard(dashboard_name)
        for idx, card_params in enumerate(cards):
            card_id = self.create_card(**card_params)
            self.add_card_to_dashboard(dashboard_id, card_id, row=0, col=idx*4)
        return dashboard_id

In [ ]:
generator = MetabaseDashboardGenerator(
    metabase_url="http://localhost:3000",
    username="a.adamovich@just-ai.com",
    password="123cat123"
)

In [ ]:

# Пример создания карты (SQL-запрос)
card_params = {
    "name": "Total Sales by Day 3",
    "query": {
        "type": "native",
        "native": {
            "query": "SELECT timestamp::date as date_day, sum(amount) as total_sales FROM orders GROUP BY timestamp::date"
        },
        "database": 2
    },
    "display": "bar",
    "visualization_settings": {
        "x_axis": "date_day",
        "y_axis": "total_sales"
  }
}
card_id = generator.create_card(**card_params)

print(f"Метрики создана! ID: {card_id}")

In [ ]:
generator._headers()

In [ ]:
req = requests.post("http://localhost:3000/api/cards/move",
    headers={
      "Content-Type": "application/json",
      'X-Metabase-Session': '3fa8eb0d-2341-490f-86be-266669f413a6'
    },
    json={
      "card_ids": [
        65, 66
      ],
      "collection_id": 1,
      "dashboard_id": 33
    })

In [ ]:
req.raise_for_status()

In [ ]:
dashboard_id = generator.create_dashboard(
    name="Test",    
)

print(f"Дашборд создан! ID: {dashboard_id}")

In [ ]:
generator.get_dashboard(33)

In [ ]:
generator.add_card_to_dashboard(dashboard_id=33,
                                card_id=65,
                                col=1,
                                row=1,
                                sizeX=10,
                                sizeY=10)
print(f"Метрика добавлена на дашборд")

In [ ]:
new_dashcard = {
            "col": 1,
            "id": 33,
            "parameter_mappings": [
                {
                    "parameter_id": "",
                    "target": None
                }
            ],
            "row": 1,
            "series": [
                {}
            ],
            "size_x": 4,
            "size_y": 4
        }

example = {'description': '',
 'archived': False,
 'view_count': 10,
 'collection_position': None,
 'dashcards': [
     new_dashcard
 ],
 'param_values': None,
 'initially_published_at': None,
 'can_write': True,
 'tabs': [],
 'enable_embedding': False,
 'collection_id': None,
 'show_in_getting_started': False,
 'name': 'Test',
 'width': 'fixed',
 'caveats': None,
 'collection_authority_level': None,
 'creator_id': 1,
 'can_restore': False,
 'updated_at': '2025-05-22T07:48:49.110547Z',
 'made_public_by_id': None,
 'embedding_params': None,
 'cache_ttl': None,
 'last_used_param_values': {},
 'id': 33,
 'last_viewed_at': '2025-05-22T07:48:49.110547Z',
 'position': None,
 'archived_directly': False,
 'entity_id': 'z-Bqta381yTBePUiLlmyI',
 'param_fields': None,
 'last-edit-info': {'id': 1,
  'email': 'a.adamovich@just-ai.com',
  'first_name': 'Anatoly',
  'last_name': 'Adamovich',
  'timestamp': '2025-05-22T07:48:49.15732Z'},
 'collection': {'metabase.models.collection.root/is-root?': True,
  'authority_level': None,
  'name': 'Наша статистика',
  'is_personal': False,
  'id': 'root',
  'can_write': True},
 'parameters': [],
 'auto_apply_filters': True,
 'created_at': '2025-05-22T07:48:49.110547Z',
 'public_uuid': None,
 'points_of_interest': None,
 'can_delete': False
}

In [ ]:
generator.get_dashboard(33)

In [ ]:
url = "http://localhost:3000/api/dashboard/33"
response = requests.put(url, json=example, headers=generator._headers())
response.raise_for_status()